# KoBERT Tutorial

YBIGTA 9기 박성남 (sungnam1108@naver.com)

---
BERT는 한줄로 말하자면 대규모의 unlabeled 텍스트 데이터로 좋은 sentence representation을 사전학습한 다음에 이를 바탕으로 downstream task를 푸는 모델입니다. 워낙 성능이 좋고 패키징이 잘되있어요. SKT T-Brain에서 공개한 KoBERT 모델(한국어 버전 BERT)을 가지고 간단한 emoji classification을 해보겠습니다.

#### 패키지 설치

In [2]:
!pip install -r ../requirements.txt

  Cloning https://git@github.com/SKTBrain/KoBERT.git (to revision master) to /private/var/folders/dn/3htq47qn1qq3ww4sgyywysj40000gn/T/pip-req-build-cwtawug5
  Running command git clone -q https://git@github.com/SKTBrain/KoBERT.git /private/var/folders/dn/3htq47qn1qq3ww4sgyywysj40000gn/T/pip-req-build-cwtawug5
     |████████████████████████████████| 627.7MB 79kB/s 
     |████████████████████████████████| 245kB 3.4MB/s 
     |████████████████████████████████| 1.1MB 3.3MB/s 
     |████████████████████████████████| 317kB 3.8MB/s 
     |████████████████████████████████| 71.1MB 828kB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.8.2-cp36-none-any.whl size=293515 sha256=e763d07fc2fad508cc67e20874b5ccf4b98b4f3887d148f591a519ddb8eca309
  Stored in directory: /Users/angrypark/Library/Caches/pip/wheels/a4/aa/61/0aebc5c078c4b1ccf325cd7579932b99403008da6e7ce6b68f
  Created wheel for kobert: filename=kobert-0.1.1-cp36-none-any.whl size=12854 sha256=293d4e86539e67f646723935eab557f4c98e8353062e

#### 라이브러리

In [4]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [5]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

I1227 11:37:21.860177 4409351488 file_utils.py:39] PyTorch version 1.3.1 available.
I1227 11:37:22.100258 4409351488 modeling_xlnet.py:194] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


In [6]:
from transformers import AdamW
from transformers.optimization import WarmupLinearSchedule

#### BERT 모델 불러오기

In [7]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")

In [9]:
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


#### 데이터셋과 Tokenizer 불러오기

일단 데이터셋 좀 원하는 형태로 바꾸겠습니다 

In [25]:
with open('../data/unicode.csv', 'r') as f:
    label2emoji = [line.strip() for line in f if line.strip()]
    emoji2label = {e: i for i, e in enumerate(label2emoji)}

In [44]:
with open('../data/Sample_utf.csv', 'r') as f:
    train = f.readlines()
    
with open('../data/train.txt', 'w') as f:
    for line in train[1:]:
        if not line.strip():
            continue
        reversed_line = list(reversed(line.strip().split(',')))
        reversed_line = [item.strip() for item in reversed_line]
        reversed_line[1] = str(emoji2label[reversed_line[1]])
        f.write('\t'.join(reversed_line) + '\n')

with open('../data/val.txt', 'w') as f:
    for line in train[1:]:
        if not line.strip():
            continue
        reversed_line = list(reversed(line.strip().split(',')))
        reversed_line = [item.strip() for item in reversed_line]
        reversed_line[1] = str(emoji2label[reversed_line[1]])
        f.write('\t'.join(reversed_line) + '\n')

완성된 데이터 형태는 다음과 같음

In [73]:
with open('../data/train.txt') as f:
    for _ in range(10):
        print(f.readline().strip())

움찔 수염이 바짝 곤두선다 따 따딱히 맘에 들어서 가져왔다기보단 그냥 치워주려고 했는데 자꾸 쳐다보게 되서 요 미안해요 버리는 건 줄 알았습니다만 원래 사용할 목적이 있다면 가져가도 좋답니다	22
정말 수고했어요	0
오늘 꿈에서 여행을갔는데 거기서 아이패드를 깨먹었다 넘나 처참하게 나 어제 패드케이스 주문했는데 왜 그런 꿈을 꾼거야	40
앗 많이는 안자르셨네요  제가 눈썰미가 없는건가	2
어제저녁종대시 챙겨주셔서 감사했어요 전어제종대와 기분좋은 외출을하고 돌아왔었어요 눈뜨시고 이쁜종대보시면서 오늘하루도 행복함 가득하시길 바라요	58
투마링님도 새해 복 마니마니마니 받으세요 사진 너무 잘 보고있어요	18
엇 반모버스 아까 출발했는데 반모 타임머신타죠 뭐  그래서 구구님은 몇살	44
헐 그럼 더 이뿌게 이쁜 겸딩 규리	26
39 윤기시  윤기 얼굴 못본 지 참 오래된 것 같아 너무너무 보고싶어 사랑해 민윤기	12
성우님들 완전 잘생겼어 사쿠마레성우님 진짜 잘생겼네ㅠㅠㅜㅜㅜㅠ	56


---
전체 데이터에 대해서 위와 같은 형식으로 만드시고 저장해주세요~

In [45]:
train = nlp.data.TSVDataset("../data/train.txt", field_indices=[0, 1], num_discard_samples=0)
val = nlp.data.TSVDataset("../data/train.txt", field_indices=[0, 1], num_discard_samples=0)

In [46]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


#### BertDataset 정의

In [47]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

#### 하이퍼파라미터 정의

In [61]:
num_classes = 64
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

#### Dataloader 정의

In [49]:
train = BERTDataset(train, 0, 1, tok, max_len, True, False)
val = BERTDataset(val, 0, 1, tok, max_len, True, False)

In [51]:
train_dataloader = torch.utils.data.DataLoader(train, batch_size=batch_size, num_workers=5)
val_dataloader = torch.utils.data.DataLoader(val, batch_size=batch_size, num_workers=5)

#### Fine-tuning을 위한 Classifier 정의

In [63]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=num_classes,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [64]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#### Optimizer, Learning Rate, Loss, Scheduler 정의

loss는 cross entropy loss, scheduler는 step별로 learning rate 조정해주는 친구입니다

In [65]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [66]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [67]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [68]:
scheduler = WarmupLinearSchedule(optimizer, warmup_steps=warmup_step, t_total=t_total)

#### Metric 정의

In [69]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

#### 학습

cpu에서는 안돌아가네요, gcp에서 돌려보시고 궁금한거 잇으면 톡주세요 (colab에서는 확인했습니다)

In [70]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

epoch 1 batch id 1 loss 4.218592643737793 train acc 0.0


KeyboardInterrupt: 